<a href="https://colab.research.google.com/github/MahdiTheGreat/Intro-to-language-modeling/blob/main/Retrieval_augmented_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
generative_model="meta-llama/Llama-3.2-1B-Instruct"
sentence_model = "dmis-lab/biobert-base-cased-v1.1"

In [2]:
from google.colab import userdata

In [3]:
!pip install langchain_huggingface
!pip install -qU langchain-text-splitters
!pip install -qU "langchain-chroma>=0.1.2"
!pip install -U langchain-community
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 13.2 MB/s eta 0:0

In [4]:
!wget https://raw.githubusercontent.com/pubmedqa/pubmedqa/refs/heads/master/data/ori_pqal.json

--2024-12-15 09:15:10--  https://raw.githubusercontent.com/pubmedqa/pubmedqa/refs/heads/master/data/ori_pqal.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2584787 (2.5M) [text/plain]
Saving to: ‘ori_pqal.json’

ori_pqal.json       100%[===================>]   2.46M  --.-KB/s    in 0.06s   

2024-12-15 09:15:11 (41.9 MB/s) - ‘ori_pqal.json’ saved [2584787/2584787]



We collect two datasets:

    ‘questions’: the questions with corresponding gold long answer, gold document ID, and year.
    ‘documents’: the abstracts (contexts+long_answer concatenated), and year.


In [5]:
import pandas as pd
tmp_data = pd.read_json("/content/ori_pqal.json").T
# some labels have been defined as "maybe", only keep the yes/no answers
tmp_data = tmp_data[tmp_data.final_decision.isin(["yes", "no"])]

documents = pd.DataFrame({"abstract": tmp_data.apply(lambda row: (" ").join(row.CONTEXTS+[row.LONG_ANSWER]), axis=1),
             "year": tmp_data.YEAR})
questions = pd.DataFrame({"question": tmp_data.QUESTION,
             "year": tmp_data.YEAR,
             "gold_label": tmp_data.final_decision,
             "gold_context": tmp_data.LONG_ANSWER,
             "gold_document_id": documents.index})

For an example of a query:

In [6]:
questions.iloc[0].question

'Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?'

For an example of a document to leverage for the queries:

In [7]:
text = documents.iloc[0].abstract

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings

model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=sentence_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [9]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [10]:
from langchain.text_splitter import NLTKTextSplitter
import re

# Initialize the NLTKTextSplitter
text_splitter = NLTKTextSplitter(chunk_size=1000, chunk_overlap=200)

# Split the text
chunks = text_splitter.split_text(text)

chunks = [re.sub(r'\s+', ' ', chunk) for chunk in chunks]

# Output the chunks
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {chunk}\n")

Chunk 1: Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants. The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and cells in late stages of PCD (LPCD).

Chunk 2: Window stage leaves were stained with the mitochondrial dye MitoTr

In [11]:
print(chunks[0])
print(chunks[1])

Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants. The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and cells in late stages of PCD (LPCD).
Window stage leaves were stained with the mitochondrial dye MitoTracker Red CMXRos an

In [12]:
# Generate unique ids for the chunks
chunk_ids = [i for i in range(len(chunks))]

If you want to get best in-class automated tracing of your model calls you can also set your LangSmith API key by uncommenting below:

In [13]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

In [14]:
from uuid import uuid4
from langchain_core.documents import Document

using_chunks = True
docs=[]
split_idx = 10

if using_chunks:
  ids = chunk_ids

  for i, doc in enumerate(chunks[:split_idx]):
    temp_doc = Document(
        page_content=doc,
        metadata={"source": "tweet"},
        id=ids[i],
    )
    docs.append(temp_doc)

else:

  ids = documents.iloc[:split_idx].index

  for i, doc in enumerate(documents.iloc[:split_idx].values):
    temp_doc = Document(
        page_content=doc[0],
        metadata={"source": "tweet"},
        id=ids[i],
    )
    docs.append(temp_doc)

In [15]:
from langchain.vectorstores import Chroma

try:
  vector_store.delete_collection() # Delete vector_store collection if it already exists
except:
  pass # Ignore if it doesnt exist

vector_store = Chroma.from_documents(docs, hf)
#retriever = vector_store.as_retriever(...)

In [16]:
print(vector_store._collection.count())

3


In [17]:
results = vector_store.similarity_search_with_score(
    "What is programmed cell death?", k=10
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=40.248814] Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants. The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and cells in late stages of PCD (LPCD). [{'source': 'tweet'}]
* [SIM=44.876183] Window stage leaves were st

In [18]:
from huggingface_hub import login
login(userdata.get('huggingface_token'))

In [19]:
! huggingface-cli download $generative_model --local-dir $generative_model

Fetching 13 files:   0% 0/13 [00:00<?, ?it/s]Downloading 'README.md' to 'meta-llama/Llama-3.2-1B-Instruct/.cache/huggingface/download/README.md.b3c62ef6190712fb3e31af1628f0b85706fc4ba8.incomplete'

README.md:   0% 0.00/41.7k [00:00<?, ?B/s]

LICENSE.txt:   0% 0.00/7.71k [00:00<?, ?B/s]


config.json: 100% 877/877 [00:00<00:00, 6.36MB/s]
LICENSE.txt: 100% 7.71k/7.71k [00:00<00:00, 7.46MB/s]
Download complete. Moving file to meta-llama/Llama-3.2-1B-Instruct/LICENSE.txt
Download complete. Moving file to meta-llama/Llama-3.2-1B-Instruct/config.json
README.md: 100% 41.7k/41.7k [00:00<00:00, 5.84MB/s]
Download complete. Moving file to meta-llama/Llama-3.2-1B-Instruct/README.md

USE_POLICY.md:   0% 0.00/6.02k [00:00<?, ?B/s]

USE_POLICY.md: 100% 6.02k/6.02k [00:00<00:00, 6.36MB/s]
.gitattributes: 100% 1.52k/1.52k [00:00<00:00, 5.02MB/s]
Download complete. Moving file to meta-llama/Llama-3.2-1B-Instruct/USE_POLICY.md
Download complete. Moving file to meta-llama/Llama-3.2-1B-Instruct/.gitattrib

In [81]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(generative_model)
model_hf = AutoModelForCausalLM.from_pretrained(generative_model)

In [94]:
from langchain.schema import BaseOutputParser
from tqdm import tqdm

from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

template = """Answer the question, only with yes or no, based only on the following context:
{context}
Do not write anything else. Do not explain your answer.
Question: {question}
"""

# # Define the prompt template, embedding the expected format
# prompt_template = PromptTemplate(
#     template=template,
#     input_variables=["context", "question"],
#     partial_variables={"format_instructions": output_parser.get_format_instructions()},
# )

# # Define the schema for the structured output
# response_schemas = [
#     ResponseSchema(name="Human", description="The human input to the LLM"),
#     ResponseSchema(name="Document", description="The document or context provided"),
#     ResponseSchema(name="Question", description="The question asked to the model"),
#     ResponseSchema(name="Answer", description="The answer provided by the model"),
# ]

# # Create the parser
# output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# output_parser.get_format_instructions()

prompt = ChatPromptTemplate.from_template(template)

retriever = vector_store.as_retriever()

# Create a pipeline for text generation
pipe = pipeline(
    "text-generation",
    model=model_hf,
    tokenizer=tokenizer,
    max_new_tokens=10,
    device="cuda"
    )

# Wrap the pipeline in a LangChain HuggingFacePipeline object
llm = HuggingFacePipeline(pipeline=pipe)
input={"question": "Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?"}
model_output = llm.invoke(input['question'])
print(model_output)  # Check what the raw output looks like

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Do mitochondria play a role in remodelling lace plant leaves during programmed cell death? 
A) Yes
B) No
C


In [106]:
retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = retrieval_chain.invoke(input['question'])
response

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


"Human: Answer the question, only with yes or no, based only on the following context:\n[Document(metadata={'source': 'tweet'}, page_content='Results depicted mitochondrial dynamics in vivo as PCD progresses within the lace plant, and highlight the correlation of this organelle with other organelles during developmental PCD. To the best of our knowledge, this is the first report of mitochondria and chloroplasts moving on transvacuolar strands to form a ring structure surrounding the nucleus during developmental PCD. Also, for the first time, we have shown the feasibility for the use of CsA in a whole plant system. Overall, our findings implicate the mitochondria as playing a critical and early role in developmentally regulated PCD in the lace plant.'), Document(metadata={'source': 'tweet'}, page_content='Window stage leaves were stained with the mitochondrial dye MitoTracker Red CMXRos and examined. Mitochondrial dynamics were delineated into four categories (M1-M4) based on characteri

In [108]:
# Initialize a counter for correct answers
import re

def find_answer(text):
    # Define a regex pattern to find "Answer: X"
    pattern = r"Answer:\s*(.*?)\s*(?=\n|$)"  # Match "Answer: " followed by any text until newline or end of string

    # Search for the pattern in the text
    match = re.search(pattern, text)

    if match:
        return match.group(1)  # Return the answer found
    else:
        return None  # No answer found

correct_count = 0
q_a = zip(questions.question, questions.gold_label)
print(len(questions.question))

for question, gold_label in q_a:
    response = retrieval_chain.invoke(question)
    answer = find_answer(response)
    print(f"Found answer: {answer}")
    print(f"Gold answer: {gold_label}")

    if answer is None:
        print("No answer found")
        print(response)
        continue

    if gold_label.lower() in answer.lower():
        print("Correct")
        correct_count += 1
    else:
        print("Incorrect")
        print(response)

# Print the total number of correct answers
print(f"Total correct answers: {correct_count}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


890
Found answer: None
Gold answer: yes
No answer found
Human: Answer the question, only with yes or no, based only on the following context:
[Document(metadata={'source': 'tweet'}, page_content='Results depicted mitochondrial dynamics in vivo as PCD progresses within the lace plant, and highlight the correlation of this organelle with other organelles during developmental PCD. To the best of our knowledge, this is the first report of mitochondria and chloroplasts moving on transvacuolar strands to form a ring structure surrounding the nucleus during developmental PCD. Also, for the first time, we have shown the feasibility for the use of CsA in a whole plant system. Overall, our findings implicate the mitochondria as playing a critical and early role in developmentally regulated PCD in the lace plant.'), Document(metadata={'source': 'tweet'}, page_content='Window stage leaves were stained with the mitochondrial dye MitoTracker Red CMXRos and examined. Mitochondrial dynamics were delin

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Found answer: None
Gold answer: no
No answer found
Human: Answer the question, only with yes or no, based only on the following context:
[Document(metadata={'source': 'tweet'}, page_content='Window stage leaves were stained with the mitochondrial dye MitoTracker Red CMXRos and examined. Mitochondrial dynamics were delineated into four categories (M1-M4) based on characteristics including distribution, motility, and membrane potential (ΔΨm). A TUNEL assay showed fragmented nDNA in a gradient over these mitochondrial stages. Chloroplasts and transvacuolar strands were also examined using live cell imaging. The possible importance of mitochondrial permeability transition pore (PTP) formation during PCD was indirectly examined via in vivo cyclosporine A (CsA) treatment. This treatment resulted in lace plant leaves with a significantly lower number of perforations compared to controls, and that displayed mitochondrial dynamics similar to that of non-PCD cells. Results depicted mitochondrial

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Found answer: No
Gold answer: no
Correct


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Found answer: No
Gold answer: no
Correct


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Found answer: Yes
Gold answer: yes
Correct
Found answer: No
Gold answer: no
Correct
Total correct answers: 415


In [110]:
print(f"Accuracy: {correct_count / len(questions.question)}")

Accuracy: 0.46629213483146065
